In [1]:
import os
import json
import shutil
import subprocess
import time
import random

from pydub import AudioSegment
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
import numpy as np
from IPython.display import Audio

def beep(frequency=2500.0, fs=44100):
    frequency = 1000  # Frequency of the beep in Hz
    duration = 10    # Total duration of the sound pattern in seconds
    sample_rate = 44100  # Sampling rate in Hz
    on_duration = 0.25  # Duration of the beep in seconds
    off_duration = 0.5  # Duration of silence in seconds
    
    # Generate a single cycle of sound and silence
    t_on = np.linspace(0, on_duration, int(sample_rate * on_duration), False)
    beep = np.sin(2 * np.pi * frequency * t_on)
    silence = np.zeros(int(sample_rate * off_duration))
    
    # Concatenate sound and silence to create the pattern
    single_cycle = np.concatenate((beep, silence))
    cycles = int(duration / (on_duration + off_duration))
    sound_pattern = np.tile(single_cycle, cycles)
    
    # Ensure the sound is loud enough by normalizing its amplitude
    sound_pattern *= 0.1 / np.max(np.abs(sound_pattern))
    display(Audio(sound_pattern, rate=fs, autoplay=True))

In [3]:
# dataset = "dataset-dummy"
# print(os.listdir(dataset))
# shutil.rmtree(dataset + "-processed")
# os.mkdir(dataset + "-processed")

In [4]:
def extract_words_and_timestamps(caption_file_path):
    with open(caption_file_path, 'r') as caption_file:
        caption = json.load(caption_file)
        events = caption['events']
        all_words = []
        all_timestamps = []
        for event in events:
            if 'segs' not in event or event['segs'][0]['utf8'] == '\n':
                continue
            words = [x['utf8'].strip(' ') for x in event['segs']]
            timestamps = [event['tStartMs'] + (x.get('tOffsetMs', 0)) for x in event['segs']]
            all_words += words
            all_timestamps += timestamps
    return all_words, all_timestamps

## Base

In [9]:
# def run_ffmpeg(mp4_path, segment, idx, segments_path):
#     cmd = ['ffmpeg', '-y', '-i', mp4_path, '-ss', str(segment[0] / 1000), '-to', str(segment[1] / 1000), '-c:a', 'flac', segments_path + '/segment_' + str(idx) + '.flac']
#     subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

In [10]:
# folders = os.listdir(dataset)
# total_segmenting_time = 0
# total_encoding_time = 0
# total_segments = 0

# for folder in folders:
#     print("\nWorking", folder)

#     mp4_path = dataset + "/" + folder + "/audio.mp4"
#     caption_path = dataset + "/" + folder + "/caption-a.en.json"
#     segments_path = dataset + "-processed/" + folder + "/segments"

#     # Step 1: Filter only for ones with mp4 and a-en captions
#     if not os.path.exists(mp4_path):
#         print("No mp4 file exists. Skipping.")
#         continue
#     if not os.path.exists(caption_path):
#         print("No automated english caption exists. Skipping.")
#         continue

#     # Step 2: Get the segments
#     start_time = time.time()
#     words, timestamps = extract_words_and_timestamps(caption_path)
#     segments = []
#     next_start_ts = 0
#     next_start_index = 0
    
#     for i, timestamp in enumerate(timestamps):
#         if timestamp < next_start_ts + 8000:
#             continue
#         if i < 2:
#             continue
#         segments.append((next_start_ts, (timestamps[i - 1] + timestamp) / 2, next_start_index, i - 1))
#         #next_start_ts = (timestamps[i - 1] + timestamp) / 2
#         next_start_ts = timestamp
#         next_start_index = i
#     end_time = time.time()
#     total_segmenting_time += end_time - start_time
#     total_segments += len(segments)

#     start_time = time.time()
#     # Step 3: Chop into segments
#     if not os.path.exists(segments_path):
#         os.mkdir(dataset + "-processed/" + folder)
#         os.mkdir(segments_path)

#     with ThreadPoolExecutor(max_workers=48) as executor:
#         for idx, segment in enumerate(segments):
#             executor.submit(run_ffmpeg, mp4_path, segment, idx, segments_path)

#     end_time = time.time()
#     total_encoding_time += (end_time - start_time)


Working D3owqp72SJE

Working R9vCEIjnEMQ

Working orJSJGHjBLI
No automated english caption exists. Skipping.

Working mj0XInqZMHY
No automated english caption exists. Skipping.

Working zz1t1pdlzic

Working N6D0GsSTSOk

Working ZCCPjB_57mM

Working sAS7Hzgxdvc

Working YvnTgfN7e4s

Working znBfBah3hMY

Working 0hBfDMtYM3s
No mp4 file exists. Skipping.

Working 0hBxfFw87zo
No automated english caption exists. Skipping.

Working 0HlqdedzU3M

Working 0IBJwcvjZIY

Working 0ik6X4DJKCc

Working 0ILjxr1hsBo

Working 0IlqCm9r4tA

Working 0IRE93YIyFs
No mp4 file exists. Skipping.

Working 0iRvEPcQV3I
No mp4 file exists. Skipping.

Working 0Itr1xiEigI

Working 0Jf1l5drcmo
No mp4 file exists. Skipping.

Working 0jGX1yM1pnM

Working 0JIiM1EegJU

Working 0jjeOYMjmDU

Working 0JLTVdHl2O8

Working 0jRGyE3RD1E
No mp4 file exists. Skipping.

Working 0jwm9YWm6nk

Working 0k7-Ay2fqvA

Working 0k447K0zx5I
No mp4 file exists. Skipping.

Working 0KgPwq-Ikvw


In [12]:
# print("Total segments", total_segments)
# print("Total segmenting time", total_segmenting_time)
# print("Total encoding time", total_encoding_time)
# print("Total time", total_segmenting_time + total_encoding_time)

Total segments 2561
Total segmenting time 0.18833470344543457
Total encoding time 203.0803680419922
Total time 203.26870274543762


## Encode Entire File then Split Pieces

In [13]:
# def ffmpeg_split_no_encode(flac_path, segment, idx, segments_path):
#     cmd = ['ffmpeg', '-y', '-i', flac_path, '-ss', str(segment[0] / 1000), '-to', str(segment[1] / 1000), '-c', 'copy', segments_path + '/segment_' + str(idx) + '.flac']
#     subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

In [14]:
# folders = os.listdir(dataset)
# total_segmenting_time = 0
# total_encoding_time = 0
# total_segments = 0

# for folder in folders:
#     print("\nWorking", folder)

#     mp4_path = dataset + "/" + folder + "/audio.mp4"
#     flac_path = dataset + "/" + folder + "/audio.flac"
#     caption_path = dataset + "/" + folder + "/caption-a.en.json"
#     segments_path = dataset + "-processed/" + folder + "/segments"

#     # Step 1: Filter only for ones with mp4 and a-en captions
#     if not os.path.exists(mp4_path):
#         print("No mp4 file exists. Skipping.")
#         continue
#     if not os.path.exists(caption_path):
#         print("No automated english caption exists. Skipping.")
#         continue

#     # Step 2: Get the segments
#     start_time = time.time()
#     words, timestamps = extract_words_and_timestamps(caption_path)
#     segments = []
#     next_start_ts = 0
#     next_start_index = 0
    
#     for i, timestamp in enumerate(timestamps):
#         if timestamp < next_start_ts + 8000:
#             continue
#         if i < 2:
#             continue
#         segments.append((next_start_ts, (timestamps[i - 1] + timestamp) / 2, next_start_index, i - 1))
#         #next_start_ts = (timestamps[i - 1] + timestamp) / 2
#         next_start_ts = timestamp
#         next_start_index = i
#     end_time = time.time()
#     total_segmenting_time += end_time - start_time
#     total_segments += len(segments)

#     # Step 3: Chop into segments
#     start_time = time.time()
#     cmd = ['ffmpeg', '-y', '-i', mp4_path, '-c:a', 'flac', flac_path]
#     subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    
#     if not os.path.exists(segments_path):
#         os.mkdir(dataset + "-processed/" + folder)
#         os.mkdir(segments_path)

#     with ThreadPoolExecutor(max_workers=48) as executor:
#         for idx, segment in enumerate(segments):
#             executor.submit(ffmpeg_split_no_encode, flac_path, segment, idx, segments_path)

#     end_time = time.time()
#     total_encoding_time += (end_time - start_time)


Working D3owqp72SJE

Working R9vCEIjnEMQ

Working orJSJGHjBLI
No automated english caption exists. Skipping.

Working mj0XInqZMHY
No automated english caption exists. Skipping.

Working zz1t1pdlzic

Working N6D0GsSTSOk

Working ZCCPjB_57mM

Working sAS7Hzgxdvc

Working YvnTgfN7e4s

Working znBfBah3hMY

Working 0hBfDMtYM3s
No mp4 file exists. Skipping.

Working 0hBxfFw87zo
No automated english caption exists. Skipping.

Working 0HlqdedzU3M

Working 0IBJwcvjZIY

Working 0ik6X4DJKCc

Working 0ILjxr1hsBo

Working 0IlqCm9r4tA

Working 0IRE93YIyFs
No mp4 file exists. Skipping.

Working 0iRvEPcQV3I
No mp4 file exists. Skipping.

Working 0Itr1xiEigI

Working 0Jf1l5drcmo
No mp4 file exists. Skipping.

Working 0jGX1yM1pnM

Working 0JIiM1EegJU

Working 0jjeOYMjmDU

Working 0JLTVdHl2O8

Working 0jRGyE3RD1E
No mp4 file exists. Skipping.

Working 0jwm9YWm6nk

Working 0k7-Ay2fqvA

Working 0k447K0zx5I
No mp4 file exists. Skipping.

Working 0KgPwq-Ikvw


In [15]:
# print("Total segments", total_segments)
# print("Total segmenting time", total_segmenting_time)
# print("Total encoding time", total_encoding_time)
# print("Total time", total_segmenting_time + total_encoding_time)

Total segments 2561
Total segmenting time 0.18578505516052246
Total encoding time 94.47127771377563
Total time 94.65706276893616


## Segment All Folders in Parallel

In [9]:
def ffmpeg_split_full_ref(segment):
    cmd = ['ffmpeg', '-y', '-i', segment[4], '-ss', str(segment[0] / 1000), '-to', str(segment[1] / 1000), '-c', 'copy', segment[5] + '/segment_' + str(segment[6]) + '.flac']
    # Much slower to seek first b/c of Disk I/O
    # cmd = ['ffmpeg', '-y', '-ss', str(segment[0] / 1000), '-i', segment[4], '-to', str(segment[1] / 1000), '-c', 'copy', segment[5] + '/segment_' + str(segment[6]) + '.flac']
    print(cmd)
    subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

In [17]:
# folders = os.listdir(dataset)
# total_segments = 0
# total_segmenting_time = 0
# total_encoding_time = 0
# total_spliting_time = 0

# segments = []
# for folder in folders:
#     print("\nWorking", folder)

#     mp4_path = dataset + "/" + folder + "/audio.mp4"
#     flac_path = dataset + "/" + folder + "/audio.flac"
#     caption_path = dataset + "/" + folder + "/caption-a.en.json"
#     segments_path = dataset + "-processed/" + folder + "/segments"

#     # Step 1: Filter only for ones with mp4 and a-en captions
#     if not os.path.exists(mp4_path):
#         print("No mp4 file exists. Skipping.")
#         continue
#     if not os.path.exists(caption_path):
#         print("No automated english caption exists. Skipping.")
#         continue

#     # Step 2: Get the segments
#     start_time = time.time()
#     words, timestamps = extract_words_and_timestamps(caption_path)
#     next_start_ts = 0
#     next_start_index = 0
#     seg_idx = 0
    
#     for i, timestamp in enumerate(timestamps):
#         if timestamp < next_start_ts + 8000:
#             continue
#         if i < 2:
#             continue
#         segments.append((next_start_ts, (timestamps[i - 1] + timestamp) / 2, next_start_index, i - 1, flac_path, segments_path, seg_idx))
#         seg_idx += 1
#         #next_start_ts = (timestamps[i - 1] + timestamp) / 2
#         next_start_ts = timestamp
#         next_start_index = i
#     end_time = time.time()
#     total_segmenting_time += end_time - start_time
#     total_segments += len(segments)

#     # Step 3: Encode
#     start_time = time.time()
#     cmd = ['ffmpeg', '-y', '-i', mp4_path, '-c:a', 'flac', flac_path]
#     subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
#     end_time = time.time()
#     total_encoding_time += (end_time - start_time)

#     if not os.path.exists(segments_path):
#         os.mkdir(dataset + "-processed/" + folder)
#         os.mkdir(segments_path)

# # Step 4: Chop into Segments
# start_time = time.time()
# with ThreadPoolExecutor(max_workers=48) as executor:
#     for segment in segments:
#         executor.submit(ffmpeg_split_full_ref, segment)
# end_time = time.time()
# total_spliting_time += (end_time - start_time)


Working D3owqp72SJE

Working R9vCEIjnEMQ

Working orJSJGHjBLI
No automated english caption exists. Skipping.

Working mj0XInqZMHY
No automated english caption exists. Skipping.

Working zz1t1pdlzic

Working N6D0GsSTSOk

Working ZCCPjB_57mM

Working sAS7Hzgxdvc

Working YvnTgfN7e4s

Working znBfBah3hMY

Working 0hBfDMtYM3s
No mp4 file exists. Skipping.

Working 0hBxfFw87zo
No automated english caption exists. Skipping.

Working 0HlqdedzU3M

Working 0IBJwcvjZIY

Working 0ik6X4DJKCc

Working 0ILjxr1hsBo

Working 0IlqCm9r4tA

Working 0IRE93YIyFs
No mp4 file exists. Skipping.

Working 0iRvEPcQV3I
No mp4 file exists. Skipping.

Working 0Itr1xiEigI

Working 0Jf1l5drcmo
No mp4 file exists. Skipping.

Working 0jGX1yM1pnM

Working 0JIiM1EegJU

Working 0jjeOYMjmDU

Working 0JLTVdHl2O8

Working 0jRGyE3RD1E
No mp4 file exists. Skipping.

Working 0jwm9YWm6nk

Working 0k7-Ay2fqvA

Working 0k447K0zx5I
No mp4 file exists. Skipping.

Working 0KgPwq-Ikvw


In [18]:
# print("Total segments", total_segments)
# print("Total segmenting time", total_segmenting_time)
# print("Total encoding time", total_encoding_time)
# print("Total splitting time", total_split_time)
# print("Total time", total_segmenting_time + total_encoding_time + total_spliting_time)

Total segments 26382
Total segmenting time 0.18938517570495605
Total encoding time 67.98269557952881
Total splitting time 20.477007150650024
Total time 90.8336272239685


## Encode All Folders in Parallel

In [5]:
def ffmpeg_encode(file_to_encode):
    cmd = ['ffmpeg', '-y', '-i', file_to_encode[0], '-c:a', 'flac', file_to_encode[1]]
    subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    # subprocess.run(cmd, check=True)

In [6]:
# folders = os.listdir(dataset)
# total_segments = 0
# total_segmenting_time = 0
# total_encoding_time = 0
# total_spliting_time = 0

# segments = []
# files_to_encode = []

# for folder in folders:
#     print("\nWorking", folder)

#     mp4_path = dataset + "/" + folder + "/audio.mp4"
#     flac_path = dataset + "/" + folder + "/audio.flac"
#     caption_path = dataset + "/" + folder + "/caption-a.en.json"
#     segments_path = dataset + "-processed/" + folder + "/segments"

#     # Step 1: Filter only for ones with mp4 and a-en captions
#     if not os.path.exists(mp4_path):
#         print("No mp4 file exists. Skipping.")
#         continue
#     if not os.path.exists(caption_path):
#         print("No automated english caption exists. Skipping.")
#         continue

#     # Step 2: Get the segments
#     start_time = time.time()
#     words, timestamps = extract_words_and_timestamps(caption_path)
#     next_start_ts = 0
#     next_start_index = 0
#     seg_idx = 0
    
#     for i, timestamp in enumerate(timestamps):
#         if timestamp < next_start_ts + 8000:
#             continue
#         if i < 2:
#             continue
#         segments.append((next_start_ts, (timestamps[i - 1] + timestamp) / 2, next_start_index, i - 1, flac_path, segments_path, seg_idx))
#         seg_idx += 1
#         #next_start_ts = (timestamps[i - 1] + timestamp) / 2
#         next_start_ts = timestamp
#         next_start_index = i
#     end_time = time.time()
#     total_segmenting_time += end_time - start_time
#     total_segments += len(segments)

#     files_to_encode.append((mp4_path, flac_path))

#     if not os.path.exists(segments_path):
#         os.mkdir(dataset + "-processed/" + folder)
#         os.mkdir(segments_path)

# # Step 3: Encode
# start_time = time.time()
# futures = []
# with ThreadPoolExecutor(max_workers=48) as encode_exec:
#     futures = [encode_exec.submit(ffmpeg_encode, file_to_encode) for file_to_encode in files_to_encode]
# end_time = time.time()
# for future in as_completed(futures):
#     result = future.result()
#     print(result)
# total_encoding_time += (end_time - start_time)

# # Step 4: Chop into Segments
# start_time = time.time()
# with ThreadPoolExecutor(max_workers=48) as segment_exec:
#     for segment in segments:
#         segment_exec.submit(ffmpeg_split_full_ref, segment)
# end_time = time.time()
# total_spliting_time += (end_time - start_time)


Working D3owqp72SJE

Working R9vCEIjnEMQ

Working orJSJGHjBLI
No automated english caption exists. Skipping.

Working mj0XInqZMHY
No automated english caption exists. Skipping.

Working zz1t1pdlzic

Working N6D0GsSTSOk

Working ZCCPjB_57mM

Working sAS7Hzgxdvc

Working YvnTgfN7e4s

Working znBfBah3hMY

Working 0hBfDMtYM3s
No mp4 file exists. Skipping.

Working 0hBxfFw87zo
No automated english caption exists. Skipping.

Working 0HlqdedzU3M

Working 0IBJwcvjZIY

Working 0ik6X4DJKCc

Working 0ILjxr1hsBo

Working 0IlqCm9r4tA

Working 0IRE93YIyFs
No mp4 file exists. Skipping.

Working 0iRvEPcQV3I
No mp4 file exists. Skipping.

Working 0Itr1xiEigI

Working 0Jf1l5drcmo
No mp4 file exists. Skipping.

Working 0jGX1yM1pnM

Working 0JIiM1EegJU

Working 0jjeOYMjmDU

Working 0JLTVdHl2O8

Working 0jRGyE3RD1E
No mp4 file exists. Skipping.

Working 0jwm9YWm6nk

Working 0k7-Ay2fqvA

Working 0k447K0zx5I
No mp4 file exists. Skipping.

Working 0KgPwq-Ikvw
None
None
None
None
None
None
None
None
None
None

In [7]:
# print("Total segments", total_segments)
# print("Total segmenting time", total_segmenting_time)
# print("Total encoding time", total_encoding_time)
# print("Total splitting time", total_spliting_time)
# print("Total time", total_segmenting_time + total_encoding_time + total_spliting_time)

Total segments 26382
Total segmenting time 0.1552283763885498
Total encoding time 19.528894662857056
Total splitting time 20.59140944480896
Total time 40.275532484054565


# Separate Mounted Disk with Separate Filesystem

In [7]:
dataset = "dataset-dummy"
print(os.listdir(dataset))
folders = os.listdir(dataset)


mount_point = "disk2-mount"
total_segments = 0
total_segmenting_time = 0
total_encoding_time = 0
total_spliting_time = 0

segments = []
files_to_encode = []

for folder in folders:
    print("\nWorking", folder)

    mp4_path = dataset + "/" + folder + "/audio.mp4"
    caption_path = dataset + "/" + folder + "/caption-a.en.json"

    flac_path = mount_point + "/" + dataset + "/" + folder + "/audio.flac"
    segments_path = mount_point + "/" + dataset + "-processed/" + folder + "/segments"

    # Step 1: Filter only for ones with mp4 and a-en captions
    if not os.path.exists(mp4_path):
        print("No mp4 file exists. Skipping.")
        continue
    if not os.path.exists(caption_path):
        print("No automated english caption exists. Skipping.")
        continue

    # Step 2: Get the segments
    start_time = time.time()
    words, timestamps = extract_words_and_timestamps(caption_path)
    next_start_ts = 0
    next_start_index = 0
    seg_idx = 0
    
    for i, timestamp in enumerate(timestamps):
        if timestamp < next_start_ts + 8000:
            continue
        if i < 2:
            continue
        segments.append((next_start_ts, (timestamps[i - 1] + timestamp) / 2, next_start_index, i - 1, flac_path, segments_path, seg_idx))
        seg_idx += 1
        #next_start_ts = (timestamps[i - 1] + timestamp) / 2
        next_start_ts = timestamp
        next_start_index = i
    end_time = time.time()
    total_segmenting_time += end_time - start_time

    files_to_encode.append((mp4_path, flac_path))

    if not os.path.exists(flac_path):
        os.mkdir(mount_point + "/" + dataset + "/" + folder)
    
    if not os.path.exists(segments_path):
        os.mkdir(mount_point + "/" + dataset + "-processed/" + folder)
        os.mkdir(segments_path)
total_segments = len(segments)

# Step 3: Encode
start_time = time.time()
futures = []
with ThreadPoolExecutor(max_workers=48) as encode_exec:
    futures = [encode_exec.submit(ffmpeg_encode, file_to_encode) for file_to_encode in files_to_encode]
end_time = time.time()
for future in as_completed(futures):
    result = future.result()
    print(result)
total_encoding_time += (end_time - start_time)

# Step 4: Chop into Segments
start_time = time.time()
with ThreadPoolExecutor(max_workers=48) as segment_exec:
    for segment in segments:
        segment_exec.submit(ffmpeg_split_full_ref, segment)
end_time = time.time()
total_spliting_time += (end_time - start_time)

['D3owqp72SJE', 'R9vCEIjnEMQ', 'orJSJGHjBLI', 'mj0XInqZMHY', 'zz1t1pdlzic', 'N6D0GsSTSOk', 'ZCCPjB_57mM', 'sAS7Hzgxdvc', 'YvnTgfN7e4s', 'znBfBah3hMY', '0hBfDMtYM3s', '0hBxfFw87zo', '0HlqdedzU3M', '0IBJwcvjZIY', '0ik6X4DJKCc', '0ILjxr1hsBo', '0IlqCm9r4tA', '0IRE93YIyFs', '0iRvEPcQV3I', '0Itr1xiEigI', '0Jf1l5drcmo', '0jGX1yM1pnM', '0JIiM1EegJU', '0jjeOYMjmDU', '0JLTVdHl2O8', '0jRGyE3RD1E', '0jwm9YWm6nk', '0k7-Ay2fqvA', '0k447K0zx5I', '0KgPwq-Ikvw']

Working D3owqp72SJE

Working R9vCEIjnEMQ

Working orJSJGHjBLI
No automated english caption exists. Skipping.

Working mj0XInqZMHY
No automated english caption exists. Skipping.

Working zz1t1pdlzic

Working N6D0GsSTSOk

Working ZCCPjB_57mM

Working sAS7Hzgxdvc

Working YvnTgfN7e4s

Working znBfBah3hMY

Working 0hBfDMtYM3s
No mp4 file exists. Skipping.

Working 0hBxfFw87zo
No automated english caption exists. Skipping.

Working 0HlqdedzU3M

Working 0IBJwcvjZIY

Working 0ik6X4DJKCc

Working 0ILjxr1hsBo

Working 0IlqCm9r4tA

Working 0IRE93YIyFs


In [8]:
print("Total segments", total_segments)
print("Total segmenting time", total_segmenting_time)
print("Total encoding time", total_encoding_time)
print("Total splitting time", total_spliting_time)
print("Total time", total_segmenting_time + total_encoding_time + total_spliting_time)

Total segments 2561
Total segmenting time 0.17858505249023438
Total encoding time 12.968992710113525
Total splitting time 20.545815467834473
Total time 33.69339323043823


# No ffmpeg for Splitting

In [19]:
# def load_and_chop(segments_for_one):
#     print("Calling load and chop", segments_for_one)
#     audio = AudioSegment.from_file(segments_for_one[0][4], format="flac")
#     print(audio)

#     for segment in segments_for_one:
#         audio_segment = audio[segment[0]:segment[1]]
#         audio_segment.export(segment[5] + '/segment_' + str(segment[6]) + '.flac', format="flac")

In [20]:
# dataset = "dataset-dummy"
# print(os.listdir(dataset))
# folders = os.listdir(dataset)

# mount_point = "disk2-mount"
# total_segmenting_time = 0
# total_encoding_time = 0
# total_spliting_time = 0

# segments = {}
# files_to_encode = []

# for folder in folders:
#     print("\nWorking", folder)

#     mp4_path = dataset + "/" + folder + "/audio.mp4"
#     caption_path = dataset + "/" + folder + "/caption-a.en.json"

#     flac_path = mount_point + "/" + dataset + "/" + folder + "/audio.flac"
#     segments_path = mount_point + "/" + dataset + "-processed/" + folder + "/segments"

#     # Step 1: Filter only for ones with mp4 and a-en captions
#     if not os.path.exists(mp4_path):
#         print("No mp4 file exists. Skipping.")
#         continue
#     if not os.path.exists(caption_path):
#         print("No automated english caption exists. Skipping.")
#         continue

#     # Step 2: Get the segments
#     start_time = time.time()
#     words, timestamps = extract_words_and_timestamps(caption_path)
#     next_start_ts = 0
#     next_start_index = 0
#     seg_idx = 0
    
#     for i, timestamp in enumerate(timestamps):
#         if timestamp < next_start_ts + 8000:
#             continue
#         if i < 2:
#             continue
#         if folder not in segments:
#             segments[folder] = []
#         segments[folder].append((next_start_ts, (timestamps[i - 1] + timestamp) / 2, next_start_index, i - 1, flac_path, segments_path, seg_idx))
#         seg_idx += 1
#         #next_start_ts = (timestamps[i - 1] + timestamp) / 2
#         next_start_ts = timestamp
#         next_start_index = i
#     end_time = time.time()
#     total_segmenting_time += end_time - start_time

#     files_to_encode.append((mp4_path, flac_path))

#     if not os.path.exists(flac_path):
#         os.mkdir(mount_point + "/" + dataset + "/" + folder)
    
#     if not os.path.exists(segments_path):
#         os.mkdir(mount_point + "/" + dataset + "-processed/" + folder)
#         os.mkdir(segments_path)

# # Step 3: Encode
# start_time = time.time()
# futures = []
# with ThreadPoolExecutor(max_workers=48) as encode_exec:
#     futures = [encode_exec.submit(ffmpeg_encode, file_to_encode) for file_to_encode in files_to_encode]
# end_time = time.time()
# for future in as_completed(futures):
#     result = future.result()
#     print(result)
# total_encoding_time += (end_time - start_time)

# # Step 4: Chop into Segments
# start_time = time.time()
# with ThreadPoolExecutor(max_workers=48) as split_exec:
#     futures = [split_exec.submit(load_and_chop, segments[folder]) for folder in segments]
# end_time = time.time()
# for future in as_completed(futures):
#     result = future.result()
#     print(result)
# total_spliting_time += (end_time - start_time)

['D3owqp72SJE', 'R9vCEIjnEMQ', 'orJSJGHjBLI', 'mj0XInqZMHY', 'zz1t1pdlzic', 'N6D0GsSTSOk', 'ZCCPjB_57mM', 'sAS7Hzgxdvc', 'YvnTgfN7e4s', 'znBfBah3hMY', '0hBfDMtYM3s', '0hBxfFw87zo', '0HlqdedzU3M', '0IBJwcvjZIY', '0ik6X4DJKCc', '0ILjxr1hsBo', '0IlqCm9r4tA', '0IRE93YIyFs', '0iRvEPcQV3I', '0Itr1xiEigI', '0Jf1l5drcmo', '0jGX1yM1pnM', '0JIiM1EegJU', '0jjeOYMjmDU', '0JLTVdHl2O8', '0jRGyE3RD1E', '0jwm9YWm6nk', '0k7-Ay2fqvA', '0k447K0zx5I', '0KgPwq-Ikvw']

Working D3owqp72SJE

Working R9vCEIjnEMQ

Working orJSJGHjBLI
No automated english caption exists. Skipping.

Working mj0XInqZMHY
No automated english caption exists. Skipping.

Working zz1t1pdlzic

Working N6D0GsSTSOk

Working ZCCPjB_57mM

Working sAS7Hzgxdvc

Working YvnTgfN7e4s

Working znBfBah3hMY

Working 0hBfDMtYM3s
No mp4 file exists. Skipping.

Working 0hBxfFw87zo
No automated english caption exists. Skipping.

Working 0HlqdedzU3M

Working 0IBJwcvjZIY

Working 0ik6X4DJKCc

Working 0ILjxr1hsBo

Working 0IlqCm9r4tA

Working 0IRE93YIyFs


KeyboardInterrupt: 

In [ ]:
print("Total segments", total_segments)
print("Total segmenting time", total_segmenting_time)
print("Total encoding time", total_encoding_time)
print("Total splitting time", total_spliting_time)
print("Total time", total_segmenting_time + total_encoding_time + total_spliting_time)

# Wav Instead of Flac and Drop the First

In [32]:
def ffmpeg_encode_wav(file_to_encode):
    cmd = ['ffmpeg', '-y', '-i', file_to_encode[0], '-c:a', 'pcm_s16le', '-ar', '44100', '-ac', '1', file_to_encode[1]]
    subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    # subprocess.run(cmd, check=True)

In [33]:
def ffmpeg_split_full_ref_wav(segment):
    cmd = ['ffmpeg', '-y', '-i', segment[4], '-ss', str(segment[0] / 1000), '-to', str(segment[1] / 1000), '-c', 'copy', segment[5] + '/segment_' + str(segment[6]) + '.wav']
    # Much slower to seek first b/c of Disk I/O
    # cmd = ['ffmpeg', '-y', '-ss', str(segment[0] / 1000), '-i', segment[4], '-to', str(segment[1] / 1000), '-c', 'copy', segment[5] + '/segment_' + str(segment[6]) + '.wav']
    subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    # subprocess.run(cmd, check=True)

In [38]:
dataset = "dataset-dummy-450"
print(os.listdir(dataset))
folders = os.listdir(dataset)

mount_point = "disk2-mount"
total_segments = 0
total_segmenting_time = 0
total_encoding_time = 0
total_spliting_time = 0

segments = []
files_to_encode = []

for folder in folders:
    print("\nWorking", folder)

    mp4_path = dataset + "/" + folder + "/audio.mp4"
    caption_path = dataset + "/" + folder + "/caption-a.en.json"

    wav_path = mount_point + "/" + dataset + "/" + folder + "/audio.wav"
    segments_path = mount_point + "/" + dataset + "-processed/" + folder + "/segments"

    # Step 1: Filter only for ones with mp4 and a-en captions
    if not os.path.exists(mp4_path):
        print("No mp4 file exists. Skipping.")
        continue
    if not os.path.exists(caption_path):
        print("No automated english caption exists. Skipping.")
        continue

    # Step 2: Get the segments
    start_time = time.time()
    words, timestamps = extract_words_and_timestamps(caption_path)
    next_start_ts = 0
    next_start_index = 0
    seg_idx = -1
    
    for i, timestamp in enumerate(timestamps):
        if timestamp < next_start_ts + 8000:
            continue
        if i < 2:
            continue
        if seg_idx == -1:
            seg_idx += 1
            continue
        segments.append((next_start_ts, (timestamps[i - 1] + timestamp) / 2, next_start_index, i - 1, wav_path, segments_path, seg_idx))
        seg_idx += 1
        #next_start_ts = (timestamps[i - 1] + timestamp) / 2
        next_start_ts = timestamp
        next_start_index = i
    end_time = time.time()
    total_segmenting_time += end_time - start_time

    files_to_encode.append((mp4_path, wav_path))

    if not os.path.exists(mount_point + "/" + dataset + "/" + folder):
        os.mkdir(mount_point + "/" + dataset + "/" + folder)
    
    if not os.path.exists(segments_path):
        os.mkdir(mount_point + "/" + dataset + "-processed/" + folder)
        os.mkdir(segments_path)
total_segments = len(segments)

# Step 3: Encode
start_time = time.time()
futures = []
with ThreadPoolExecutor(max_workers=48) as encode_exec:
    futures = [encode_exec.submit(ffmpeg_encode_wav, file_to_encode) for file_to_encode in files_to_encode]
end_time = time.time()
for future in as_completed(futures):
    result = future.result()
    print(result)
total_encoding_time += (end_time - start_time)

# Step 4: Chop into Segments
start_time = time.time()
with ThreadPoolExecutor(max_workers=48) as segment_exec:
    for segment in segments:
        segment_exec.submit(ffmpeg_split_full_ref_wav, segment)
end_time = time.time()
total_spliting_time += (end_time - start_time)

['Z6D4x7rdkOY', 'Z6is5YFWNHE', 'z6ktsGcffFE', 'z6ohGMgkYeE', 'z6U6yBgkiko', 'z7bM8-hcgsI', 'Z7BMdMWBXRw', 'Z7GT1vdsmMM', 'Z7Mkm1sbj90', 'Z7NXOjrKdQw', 'z7RN1Flh17c', 'z7sxmrWUW4U', 'Z7ZUzL3e3VU', 'Z8JLfr2s3DQ', 'Z8OezcEX2QY', 'z8PMT6o98eE', 'z8VFtw-b9y0', 'Z8wfBPL08mg', 'Z8xISZRzxLk', 'Z9fu_4CDNlg', 'z9fXAtrQ3Jw', 'Z9hVOCUUBvM', 'Z9kRyMe_D7g', 'z9LxT5MSCoQ', 'z9n9rqb-lvY', 'Z9oI2WXhY3c', 'z9vMczQ3jiM', 'Z9voDV_ZsB8', 'z9vrK1GU8-A', 'z9WUEVnG0CA', 'z9XWN7u8mSU', 'z12MBF6jGAQ', 'Z28Uor7z0gU', 'z35gYP4z0vE', 'Z36dw2u41BQ', 'z37QbRBXEX8', 'z44clf5atEo', 'z71xNtRTTjQ', 'Z72T16UYSxA', 'Z78Msj_77MM', 'Z78Pza15RYU', 'z81fdHR6hQ4', 'Z98svj_AwHU', 'Z199uTGrz78', 'ZaCFA5reK9E', 'zaCk515X7IM', 'Zae7hxSD3a8', 'zaf6StO4gy4', 'zaFUjkhb0bY', 'Zagp7rzjuO0', 'ZaHVCqTkZao', 'zau-F0L_hrY', 'ZAxhEUfpSss', 'ZazBiZ6ktfk', 'ZAzL-EDLDn0', 'ZB7_1CoKntI', 'zb9GYGClk8E', 'zbeGqzHKvTQ', 'zbkSFw7VkPc', 'ZbktVQyUd88', 'Zbtk0Z3jIZY', 'ZBub1q54cWE', 'zbx_YtfpbKE', 'ZBYoI7e3v5Y', 'zC-EwvxGUBU', 'Zc7IeqodhOY', 'zcb2Tmk0

In [39]:
print("Total segments", total_segments)
print("Total segmenting time", total_segmenting_time)
print("Total encoding time", total_encoding_time)
print("Total splitting time", total_spliting_time)
print("Total time", total_segmenting_time + total_encoding_time + total_spliting_time)

Total segments 46223
Total segmenting time 2.564945697784424
Total encoding time 71.65636801719666
Total splitting time 493.4151384830475
Total time 567.6364521980286


# Include the Text

In [13]:
def ffmpeg_split_full_ref_wav(segment):
    cmd = ['ffmpeg', '-y', '-i', segment[4], '-ss', str(segment[0] / 1000), '-to', str(segment[1] / 1000), '-c', 'copy', segment[5] + '/segment_' + str(segment[6]) + '.wav']
    # Much slower to seek first b/c of Disk I/O
    # cmd = ['ffmpeg', '-y', '-ss', str(segment[0] / 1000), '-i', segment[4], '-to', str(segment[1] / 1000), '-c', 'copy', segment[5] + '/segment_' + str(segment[6]) + '.wav']
    
    output_string = ' '.join(cmd)
    subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    with open("ffmpeg_commands.txt", 'a') as f:
        f.write("/usr/bin/time -v " + output_string + "\n")
    # subprocess.run(cmd, check=True)

In [14]:
def ffmpeg_encode_wav(file_to_encode):
    # Flipped to no to speed up split testing
    cmd = ['ffmpeg', '-n', '-i', file_to_encode[0], '-c:a', 'pcm_s16le', '-ar', '44100', '-ac', '1', file_to_encode[1]]
    subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    # subprocess.run(cmd, check=True)

In [17]:
dataset = "dataset-dummy"
print(os.listdir(dataset))
folders = os.listdir(dataset)

mount_point = "disk2-mount"
total_segments = 0
total_segmenting_time = 0
total_encoding_time = 0
total_spliting_time = 0

segments = []
files_to_encode = []

for folder in folders:
    # print("\nWorking", folder)

    mp4_path = dataset + "/" + folder + "/audio.mp4"
    caption_path = dataset + "/" + folder + "/caption-a.en.json"

    wav_path = mount_point + "/" + dataset + "/" + folder + "/audio.wav"
    segments_path = mount_point + "/" + dataset + "-processed/" + folder + "/segments"
    transcript_path = mount_point + "/" + dataset + "-processed/" + folder + "/segments/segment_transcript.txt"

    # Step 1: Filter only for ones with mp4 and a-en captions
    if not os.path.exists(mp4_path):
        # print("No mp4 file exists. Skipping.")
        continue
    if not os.path.exists(caption_path):
        # print("No automated english caption exists. Skipping.")
        continue

    # Step 2: Get the segments
    start_time = time.time()
    words, timestamps = extract_words_and_timestamps(caption_path)
    next_start_ts = 0
    next_start_index = 0
    seg_idx = -1

    if not os.path.exists(mount_point + "/" + dataset + "/" + folder):
        os.mkdir(mount_point + "/" + dataset + "/" + folder)
    
    if not os.path.exists(segments_path):
        os.mkdir(mount_point + "/" + dataset + "-processed/" + folder)
        os.mkdir(segments_path)

    out_file = open(transcript_path, "w")
    
    for i, timestamp in enumerate(timestamps):
        if timestamp < next_start_ts + 8000:
            continue
        if i < 2:
            continue
        if seg_idx == -1:
            seg_idx += 1
            continue
        segments.append((next_start_ts, (timestamps[i - 1] + timestamp) / 2, next_start_index, i - 1, wav_path, segments_path, seg_idx))
        segment_transcript = ' '.join(words[next_start_index:i-1])
        out_file.write("segment_" + str(seg_idx) + ".wav " + segment_transcript + "\n")
        seg_idx += 1
        #next_start_ts = (timestamps[i - 1] + timestamp) / 2
        next_start_ts = timestamp
        next_start_index = i
    
    files_to_encode.append((mp4_path, wav_path))
    out_file.close()
    
    end_time = time.time()
    total_segmenting_time += (end_time - start_time)
    

total_segments = len(segments)

# Step 3: Encode
start_time = time.time()
futures = []
with ThreadPoolExecutor(max_workers=48) as encode_exec:
    futures = [encode_exec.submit(ffmpeg_encode_wav, file_to_encode) for file_to_encode in files_to_encode]
end_time = time.time()
for future in as_completed(futures):
    result = future.result()
    print(result)
total_encoding_time += (end_time - start_time)

# Step 4: Split
start_time = time.time()
with ThreadPoolExecutor(max_workers=48) as segment_exec:
    for segment in segments:
        segment_exec.submit(ffmpeg_split_full_ref_wav, segment)
end_time = time.time()
total_spliting_time += (end_time - start_time)

['D3owqp72SJE', 'R9vCEIjnEMQ', 'orJSJGHjBLI', 'mj0XInqZMHY', 'zz1t1pdlzic', 'N6D0GsSTSOk', 'ZCCPjB_57mM', 'sAS7Hzgxdvc', 'YvnTgfN7e4s', 'znBfBah3hMY', '0hBfDMtYM3s', '0hBxfFw87zo', '0HlqdedzU3M', '0IBJwcvjZIY', '0ik6X4DJKCc', '0ILjxr1hsBo', '0IlqCm9r4tA', '0IRE93YIyFs', '0iRvEPcQV3I', '0Itr1xiEigI', '0Jf1l5drcmo', '0jGX1yM1pnM', '0JIiM1EegJU', '0jjeOYMjmDU', '0JLTVdHl2O8', '0jRGyE3RD1E', '0jwm9YWm6nk', '0k7-Ay2fqvA', '0k447K0zx5I', '0KgPwq-Ikvw']
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [18]:
print("Total segments", total_segments)
print("Total segmenting time", total_segmenting_time)
print("Total encoding time", total_encoding_time)
print("Total splitting time", total_spliting_time)
print("Total time", total_segmenting_time + total_encoding_time + total_spliting_time)

Total segments 2559
Total segmenting time 0.14357733726501465
Total encoding time 0.08951425552368164
Total splitting time 13.494420289993286
Total time 13.727511882781982


In [ ]:
beep()